In [1]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
#import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('default')
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
import os
if not os.path.exists('analysis'):
    os.makedirs('analysis')
#import plotly.express as px

In [2]:
datasets = ['(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)']
n_values = [4, 8, 16, 4, 8, 4]
n_attributes = [3, 3, 3, 4, 4, 5]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [3]:
context_unaware = False # whether original or context_unaware simulations are evaluated

setting = 'standard'
granularity_list = ['coarse', 'mixed', 'fine']
non_default_gran_list = ['coarse', 'fine']

### Determine vocab size

In [5]:
import os
import csv
from collections import Counter

# Define the directory and filename for the CSV file
output_folder = 'analysis'
csv_filename = os.path.join(output_folder, '00_vocab_sizes.csv')

# Check if the output folder exists, if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Check if the CSV file exists, if not, create it with headers
fieldnames = ['dataset', 'context_condition', 'vocab_size', 'symbols']
file_exists = os.path.exists(csv_filename)
with open(csv_filename, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if not file_exists:
        writer.writeheader()

# Open the file in append mode and write the vocab sizes for each dataset
with open(csv_filename, 'a', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Go through all datasets
    for i, d in enumerate(datasets):
        for g in granularity_list:
            # select first run
            if g != 'mixed':
                path_to_run = paths[i] + '/' + str(setting) + '/' + 'granularity_' + g + '/' + str(0) + '/'
                path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
                interaction = torch.load(path_to_interaction_train)
                
                messages = interaction.message.argmax(dim=-1)
                messages = [msg.tolist() for msg in messages]
                all_symbols = [symbol for message in messages for symbol in message]
                symbol_counts = Counter(all_symbols)
                vocab_size = len(symbol_counts)

            elif g == 'mixed':
                path_to_run = paths[i] + '/' + str(setting) + '/' + str(0) + '/'
                path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
                interaction = torch.load(path_to_interaction_train)
                
                messages = interaction.message.argmax(dim=-1)
                messages = [msg.tolist() for msg in messages]
                all_symbols = [symbol for message in messages for symbol in message]
                symbol_counts = Counter(all_symbols)
                vocab_size = len(symbol_counts)
            
            writer.writerow({'dataset': d, 'context_condition': g, 'vocab_size': vocab_size, 'symbols': symbol_counts})


In [4]:
import pandas as pd
df = pd.read_csv('analysis/00_vocab_sizes.csv')
df

,dataset,context_condition,vocab_size,symbols
0,"(3,4)",coarse,12,"Counter({0: 742, 14: 450, 9: 391, 4: 387, 6: 2..."
1,"(3,4)",mixed,15,"Counter({0: 1807, 6: 779, 2: 582, 8: 537, 11: ..."
2,"(3,4)",fine,14,"Counter({0: 742, 14: 332, 5: 315, 4: 214, 12: ..."
3,"(3,8)",coarse,17,"Counter({0: 4364, 16: 1585, 6: 1552, 17: 1485,..."
4,"(3,8)",mixed,28,"Counter({0: 11662, 25: 4555, 2: 2113, 12: 2050..."
5,"(3,8)",fine,27,"Counter({0: 4364, 23: 1292, 16: 947, 4: 872, 2..."
6,"(3,16)",coarse,32,"Counter({0: 29467, 18: 15057, 47: 10339, 41: 6..."
7,"(3,16)",mixed,52,"Counter({0: 83302, 47: 18222, 12: 13188, 37: 1..."
8,"(3,16)",fine,52,"Counter({0: 29467, 5: 8383, 22: 5202, 19: 4557..."
9,"(4,4)",coarse,15,"Counter({0: 3743, 4: 2910, 14: 2438, 15: 1895,..."


### See which messages are produced for which concepts

In [4]:
datasets = ['(3,4)',]
n_values = [4]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [8]:
# go through all datasets
for i, d in enumerate(datasets):
    for g in non_default_gran_list:
        if g != 'mixed':
        # select first run
            path_to_run = paths[i] + '/' + str(setting) +'/granularity_' + g +'/' + str(0) + '/'
        else:
            path_to_run = paths[i] + '/' + str(setting) +'/' + str(0) + '/'
            
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        print(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
        messages = [msg.tolist() for msg in messages]
        sender_input = interaction.sender_input
        print(sender_input.shape)
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[i])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
        concepts = list(zip(objects, fixed))

        # get distractor objects to re-construct context conditions
        distractor_objects = sender_input[:, n_targets:]
        distractor_objects = k_hot_to_attributes(distractor_objects, n_values[i])
        context_conds = retrieve_context_condition(objects, fixed, distractor_objects)
        #print(context_conds)

        # get random qualitative samples
        #fixed_index = random.randint(0, n_attributes[i]-1) # define a fixed index for the concept
        n_fixed = random.randint(1, n_attributes[i]) # how many fixed attributes?
        #n_fixed = 3
        fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attributes are fixed
        #fixed_indices = [0, 2, 1]
        #fixed_value = random.randint(0, n_values[i]-1) # define a fixed value for this index
        fixed_values = random.choices(range(0, n_values[i]), k=n_fixed)
        #fixed_values = [0, 1, 2]
        print(n_fixed, fixed_indices, fixed_values)
        #index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
        # TODO: adapt this loop such that multiple indices can be fixed
        all_for_this_concept = []
        for idx, (t_objects, t_fixed) in enumerate(concepts):
            #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
            if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
                for t_object in t_objects:
                    if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                        all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                        fixed = t_fixed
        #print(all_for_this_concept)                
        if len(all_for_this_concept) > 0:
            #sample = random.sample(all_for_this_concept, 20)
            sample = all_for_this_concept
            column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
            df = pd.DataFrame(sample, columns=column_names)
            df.to_csv('analysis/quali_' + str(d) + '_' + str(setting) + '_'+g +'_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv', index=False)
            print('saved ' + 'analysis/quali_' + str(d) + '_' + str(setting) +'_' + g + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv')
        else:
            raise ValueError("sample for dataset " + str(d) + g + " could not be generated")

results/(3,4)_game_size_10_vsf_3/standard/granularity_coarse/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([742, 20, 12])
1 [0] [1]
saved analysis/quali_(3,4)_standard_coarse_[1. 0. 3.],[1. 0. 0.]all.csv
results/(3,4)_game_size_10_vsf_3/standard/granularity_fine/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([742, 20, 12])
1 [2] [0]
saved analysis/quali_(3,4)_standard_fine_[2. 0. 0.],[0. 0. 1.]all.csv
results/(3,8)_game_size_10_vsf_3/standard/granularity_coarse/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([4364, 20, 24])
3 [2, 0, 1] [6, 4, 5]
saved analysis/quali_(3,8)_standard_coarse_[4. 5. 6.],[1. 1. 1.]all.csv
results/(3,8)_game_size_10_vsf_3/standard/granularity_fine/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([4364, 20, 24])
3 [2, 1, 0] [2, 3, 1]
saved analysis/quali_(3,8)_standard_fine_[1. 3. 2.],[1. 1. 1.]all.csv
results/(3,16)_game_size_10_vsf_3/standard/granularity_coarse/0/interactions/train/epoch_300/interaction_gpu0
torc

ValueError: sample for dataset (4,8)coarse could not be generated

### Count messages used for each dataset

In [9]:
import csv
import os
import torch

# Define the directory and filename for the CSV file
analysis_folder = 'analysis'
csv_filename = os.path.join(analysis_folder, 'message_counts.csv')

fieldnames = ['dataset', 'context_condition', 'unique_messages']

# Check if the CSV file exists, if not, create it with headers
if not os.path.exists(csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

# Go through all datasets
for i, d in enumerate(datasets):
    for g in granularity_list:
        if g != 'mixed':
      # select first run
            path_to_run = paths[i] + '/' + str(setting) + '/granularity_' + g + '/' + str(0) + '/'
        else:
            path_to_run = paths[i] + '/' + str(setting) +'/' + str(0) + '/'

        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        print(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
        messages = [msg.tolist() for msg in messages]
        unique_messages = set(tuple(msg) for msg in messages)  # Convert lists to tuples before adding to the set

        mess_count = len(unique_messages)
        # Check if the combination of dataset and context condition already exists in the CSV file
        combination_exists = False
        with open(csv_filename, 'r', newline='') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if row['dataset'] == d and row['context_condition'] == g:
                    combination_exists = True
                    break

        # If the combination exists, update the corresponding row
        if combination_exists:
            rows = []
            with open(csv_filename, 'r', newline='') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    if row['dataset'] == d and row['context_condition'] == g:
                        row['unique_messages'] = mess_count
                    rows.append(row)
            
            with open(csv_filename, 'w', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(rows)
        # If the combination does not exist, append a new row
        else:
            with open(csv_filename, 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow({'dataset': d, 'context_condition': g, 'unique_messages': mess_count})


results/(3,4)_game_size_10_vsf_3/standard/granularity_coarse/0/interactions/train/epoch_300/interaction_gpu0
results/(3,4)_game_size_10_vsf_3/standard/0/interactions/train/epoch_300/interaction_gpu0
results/(3,4)_game_size_10_vsf_3/standard/granularity_fine/0/interactions/train/epoch_300/interaction_gpu0
results/(3,8)_game_size_10_vsf_3/standard/granularity_coarse/0/interactions/train/epoch_300/interaction_gpu0
results/(3,8)_game_size_10_vsf_3/standard/0/interactions/train/epoch_300/interaction_gpu0
results/(3,8)_game_size_10_vsf_3/standard/granularity_fine/0/interactions/train/epoch_300/interaction_gpu0
results/(3,16)_game_size_10_vsf_3/standard/granularity_coarse/0/interactions/train/epoch_300/interaction_gpu0
results/(3,16)_game_size_10_vsf_3/standard/0/interactions/train/epoch_300/interaction_gpu0
results/(3,16)_game_size_10_vsf_3/standard/granularity_fine/0/interactions/train/epoch_300/interaction_gpu0
results/(4,4)_game_size_10_vsf_3/standard/granularity_coarse/0/interactions/tra

In [10]:
import pandas as pd
df = pd.read_csv('analysis/message_counts.csv')
df

,dataset,context_condition,unique_messages
0,"(3,4)",coarse,33
1,"(3,4)",mixed,175
2,"(3,4)",fine,133
3,"(3,8)",coarse,41
4,"(3,8)",mixed,803
5,"(3,8)",fine,438
6,"(3,16)",coarse,133
7,"(3,16)",mixed,4032
8,"(3,16)",fine,1308
9,"(4,4)",coarse,204


## Concepts and messages

### Explorative analysis

In [11]:
import sklearn

In [145]:
datasets = ['(3,4)']
n_values = [4]
n_attributes = [4]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]
setting = 'standard'

granularity = 'coarse'

In [ ]:
# go through all datasets
for i, d in enumerate(datasets):
        # select first run
        path_to_run = paths[i] + '/' + str(setting) +'/granularity_' + granularity +'/' + str(0) + '/'
            
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        interaction = torch.load(path_to_interaction_train)
        print(path_to_interaction_train)

        messages = interaction.message.argmax(dim=-1)
        messages = [msg.tolist() for msg in messages]
        sender_input = interaction.sender_input
        print(sender_input.shape)
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[i])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
        concepts = list(zip(objects, fixed))

        # get distractor objects to re-construct context conditions
        distractor_objects = sender_input[:, n_targets:]
        distractor_objects = k_hot_to_attributes(distractor_objects, n_values[i])
        context_conds = retrieve_context_condition(objects, fixed, distractor_objects)
        #print(context_conds)

        # get random qualitative samples
        #fixed_index = random.randint(0, n_attributes[i]-1) # define a fixed index for the concept
        #n_fixed = random.randint(1, n_attributes[i]) # how many fixed attributes?
        n_fixed =1
        #fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attributes are fixed
        fixed_indices = [0]
        #fixed_value = random.randint(0, n_values[i]-1) # define a fixed value for this index
        #fixed_values = random.choices(range(0, n_values[i]), k=n_fixed)
        fixed_values = [2]
        print(n_fixed, fixed_indices, fixed_values)
        #index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
        # TODO: adapt this loop such that multiple indices can be fixed
        all_for_this_concept = []
        for idx, (t_objects, t_fixed) in enumerate(concepts):
            #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
            if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
                for t_object in t_objects:
                    if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                        all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                        fixed = t_fixed
        #print(all_for_this_concept)                
        if len(all_for_this_concept) > 0:
            #sample = random.sample(all_for_this_concept, 20)
            sample = all_for_this_concept
            column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
            df = pd.DataFrame(sample, columns=column_names)
            df.to_csv('analysis/quali_' + str(d) + '_' + str(setting) + '_'+granularity +'_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv', index=False)
            print('saved ' + 'analysis/quali_' + str(d) + '_' + str(setting) +'_' + granularity+ '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv')
        else:
            raise ValueError("sample for dataset " + str(d) + granularity + " could not be generated")

### Concepts for given messages in coarse context condition

In [12]:
def create_alternative_representation(concept):
        obj, fixed = concept
        new_obj = [
            value if is_fixed else '?' 
            for value, is_fixed in zip(obj, fixed)
        ]
        rep = (((new_obj), (fixed)))

        return rep



In [40]:

def concepts_for_message(df, messages):
    concepts = []
    for message in messages:
        ex = df[df['message'] == str(message)]

        # Extract the concepts from the matching row
        concepts_str = ex.iloc[0, 1]
        # Convert the string representation of the tuple of lists to a Python object (tuple of lists)
        concepts_lists = ast.literal_eval(concepts_str)
        immutable_data = [(tuple(item1), tuple(item2)) for item1, item2 in concepts_lists]
        unique_concepts= list(immutable_data)
        for concept_tuple in unique_concepts:
            concepts.append(concept_tuple)
    return concepts

In [15]:
import torch
import pandas as pd
import random

# Assuming these variables are already defined: datasets, paths, setting, non_default_gran_list, n_epochs, n_values, n_attributes


# Go through all datasets
for i, d in enumerate(datasets):
    path_to_run = paths[i] + '/' + str(setting) + '/granularity_coarse' + '/' + str(0) + '/'
    
    path_to_interaction_train = path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0'
    path_to_interaction_val = path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0'
    interaction = torch.load(path_to_interaction_train)
    #print("Interaction train path:", path_to_interaction_train)

    messages = interaction.message.argmax(dim=-1)
    messages = [msg.tolist() for msg in messages]
    sender_input = interaction.sender_input
    #print("Sender input shape:", sender_input.shape)
    n_targets = int(sender_input.shape[1] / 2)
    
    # Get target objects and fixed vectors to re-construct concepts
    target_objects = sender_input[:, :n_targets]
    target_objects = k_hot_to_attributes(target_objects, n_values[i])
    # Concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
    (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
    concepts = list(zip(objects, fixed))
    #print(concepts[0])

   # Collect data for the DataFrame

    mess = []
    concept_reps= []

    for idx, (t_objects, t_fixed) in enumerate(concepts):
        #concept_sample = (tuple(t_objects[0]), tuple(t_fixed))
        concept_sample = (
                list(int(x) for x in t_objects[0]), 
                list(int(x) for x in t_fixed)
            )
        mess.append(tuple(messages[idx]))
        concept_reps.append(create_alternative_representation(concept_sample))

    # Create the DataFrame
    df = pd.DataFrame({'concepts': concept_reps, 'message': mess})


    # Group by messages and aggregate concepts
    message_df = df.groupby('message', as_index=False).agg({'concepts': lambda x: list(x)})
    message_df.to_csv('analysis/messages/'+str(d) + '_' + str(setting) + '_granularity_coarse.csv', index=False)
    

### D(3,4)

In [198]:
# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(4,4,4,0),]):
    print(c)

### fixed 1 is 1st postion whaaaaat

((1, 3, '?'), (1, 1, 0))
((1, 1, 2), (1, 1, 1))
((1, 0, 2), (1, 1, 1))
((1, 0, '?'), (1, 1, 0))
((1, 1, 1), (1, 1, 1))
((1, 3, 0), (1, 1, 1))
((1, 1, 3), (1, 1, 1))
((1, 0, 1), (1, 1, 1))
((1, 3, 1), (1, 1, 1))
((1, 1, '?'), (1, 1, 0))
((1, '?', 1), (1, 0, 1))
((1, 3, 3), (1, 1, 1))
((1, 3, 2), (1, 1, 1))
((1, '?', '?'), (1, 0, 0))
((1, '?', 0), (1, 0, 1))
((1, 1, 0), (1, 1, 1))
((1, 0, 0), (1, 1, 1))


In [199]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(14,14,14,0),]):
    print(c)


## it looks loke this message referes to fixed 2 in second position whaaaaat

((0, 2, 3), (1, 1, 1))
((0, 2, '?'), (1, 1, 0))
((2, 2, 0), (1, 1, 1))
(('?', 2, 1), (0, 1, 1))
((1, 2, 2), (1, 1, 1))
((2, 2, 1), (1, 1, 1))
((0, 2, 1), (1, 1, 1))
(('?', 2, 3), (0, 1, 1))
((1, 2, '?'), (1, 1, 0))
(('?', 2, '?'), (0, 1, 0))
((2, 2, '?'), (1, 1, 0))
((1, 2, 0), (1, 1, 1))
((3, 2, 1), (1, 1, 1))
((0, 2, 0), (1, 1, 1))
((3, 2, 0), (1, 1, 1))
((2, 2, 2), (1, 1, 1))
((3, 2, 2), (1, 1, 1))
((3, 2, '?'), (1, 1, 0))
((3, 2, 3), (1, 1, 1))
((1, 2, 1), (1, 1, 1))


In [200]:

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(9,9,9,0),]):
    print(c)
## it looks loke this message referes to fixed 0 in first position whaaaaat

((0, 2, 3), (1, 1, 1))
((0, 0, '?'), (1, 1, 0))
((0, 1, 3), (1, 1, 1))
((0, 1, 0), (1, 1, 1))
((0, 0, 0), (1, 1, 1))
((0, 3, 1), (1, 1, 1))
((0, 3, 0), (1, 1, 1))
((0, '?', 0), (1, 0, 1))
((0, 1, 1), (1, 1, 1))
((0, 3, 3), (1, 1, 1))
((0, 0, 2), (1, 1, 1))
((0, 1, 2), (1, 1, 1))
((0, 1, '?'), (1, 1, 0))
((0, 3, 2), (1, 1, 1))
((0, '?', 2), (1, 0, 1))
((0, '?', 1), (1, 0, 1))
((0, 3, '?'), (1, 1, 0))
((0, '?', 3), (1, 0, 1))
((0, 0, 1), (1, 1, 1))
((0, '?', '?'), (1, 0, 0))


In [286]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')

for c in concepts_for_message(df, [(15,15,15,0),]):
    print(c)

## it looks loke this message referes to fixed 0 in second position whaaaaat

((3, 0, '?'), (1, 1, 0))
((2, 0, '?'), (1, 1, 0))
((2, 0, 1), (1, 1, 1))
(('?', 0, 0), (0, 1, 1))
(('?', 0, '?'), (0, 1, 0))
((3, 0, 3), (1, 1, 1))


In [248]:
import ast
import pandas as pd

# Load DataFrame from CSV file
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(6,6,6,0),]):
    print(c)


## it looks like ALL THESE MESSAGES referes to fixed 2 in first position AND fixed 0 in second position!!!!!!!!!!!!!!!!!!!!

((2, '?', 2), (1, 0, 1))
((2, 0, 0), (1, 1, 1))
((2, 3, 0), (1, 1, 1))
((2, '?', 3), (1, 0, 1))
((2, 1, 2), (1, 1, 1))
((2, 0, '?'), (1, 1, 0))
((2, 3, '?'), (1, 1, 0))
((2, 0, 2), (1, 1, 1))
((2, 3, 1), (1, 1, 1))
((2, '?', 0), (1, 0, 1))
((2, 1, '?'), (1, 1, 0))
((2, 0, 3), (1, 1, 1))
((2, 3, 3), (1, 1, 1))
((2, 1, 0), (1, 1, 1))


In [249]:
df = pd.read_csv('analysis/messages/(3,4)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(6, 6, 15, 0), (6, 15, 6, 0),(6, 15, 15, 0)]):
    print(c)

((2, 0, '?'), (1, 1, 0))
((2, 0, 2), (1, 1, 1))
((2, 0, '?'), (1, 1, 0))
((2, 0, 3), (1, 1, 1))
((2, 0, 2), (1, 1, 1))
((2, 0, 2), (1, 1, 1))


### D(3,8)

In [252]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(23,23,23,0),]):
    print(c)
#fixed 5 in first position

((5, 0, 5), (1, 1, 1))
((5, 7, 0), (1, 1, 1))
((5, '?', 0), (1, 0, 1))
((5, 1, 0), (1, 1, 1))
((5, 3, '?'), (1, 1, 0))
((5, 2, 6), (1, 1, 1))
((5, 7, 6), (1, 1, 1))
((5, 0, 0), (1, 1, 1))
((5, 3, 4), (1, 1, 1))
((5, 2, 5), (1, 1, 1))
((5, '?', 7), (1, 0, 1))
((5, 4, 7), (1, 1, 1))
((5, 0, 6), (1, 1, 1))
((5, 3, 7), (1, 1, 1))
((5, 5, 1), (1, 1, 1))
((5, 3, 2), (1, 1, 1))
((5, 0, 1), (1, 1, 1))
((5, 6, 2), (1, 1, 1))
((5, 1, 7), (1, 1, 1))
((5, 2, 1), (1, 1, 1))
((5, 7, 1), (1, 1, 1))
((5, 0, 2), (1, 1, 1))
((5, 4, 5), (1, 1, 1))
((5, 7, 4), (1, 1, 1))
((5, 6, 5), (1, 1, 1))
((5, 0, '?'), (1, 1, 0))
((5, 5, '?'), (1, 1, 0))
((5, 1, 3), (1, 1, 1))
((5, '?', 5), (1, 0, 1))
((5, 6, 7), (1, 1, 1))
((5, 2, '?'), (1, 1, 0))
((5, 0, 7), (1, 1, 1))
((5, 5, 7), (1, 1, 1))
((5, 3, 3), (1, 1, 1))
((5, 1, 5), (1, 1, 1))
((5, 0, 4), (1, 1, 1))
((5, 6, 1), (1, 1, 1))
((5, 1, 4), (1, 1, 1))
((5, 7, '?'), (1, 1, 0))
((5, 4, 1), (1, 1, 1))
((5, 7, 7), (1, 1, 1))
((5, 3, 5), (1, 1, 1))
((5, 6, 3), (1, 1,

In [253]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(12,12,12,0),]):
    print(c)
# fixed 2 in second position

(('?', 2, 3), (0, 1, 1))
(('?', 2, 4), (0, 1, 1))
((5, 2, 3), (1, 1, 1))
((3, 2, '?'), (1, 1, 0))
(('?', 2, '?'), (0, 1, 0))
(('?', 2, 6), (0, 1, 1))
(('?', 2, 2), (0, 1, 1))
(('?', 2, 7), (0, 1, 1))
(('?', '?', 3), (0, 0, 1))
((3, 2, 3), (1, 1, 1))
(('?', 2, 0), (0, 1, 1))
(('?', 2, 1), (0, 1, 1))
((0, 2, 7), (1, 1, 1))
((0, 2, 0), (1, 1, 1))
((0, 2, 1), (1, 1, 1))


In [255]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(23,12,12,0)]):
    print(c)
#works!

((5, 2, 3), (1, 1, 1))


In [256]:
df = pd.read_csv('analysis/messages/(3,8)_standard_granularity_coarse.csv')
for c in concepts_for_message(df, [(5,21,21,0)]):
    print(c)
 #1 in second pos

(('?', 1, 7), (0, 1, 1))
((7, 1, 4), (1, 1, 1))
(('?', 1, '?'), (0, 1, 0))
((7, 1, 3), (1, 1, 1))
((7, 1, '?'), (1, 1, 0))
((7, 1, 6), (1, 1, 1))
((7, 1, 2), (1, 1, 1))
((7, 1, 5), (1, 1, 1))
((7, 1, 0), (1, 1, 1))
(('?', 1, 6), (0, 1, 1))
(('?', 1, 5), (0, 1, 1))
(('?', 1, 1), (0, 1, 1))
((7, 1, 7), (1, 1, 1))


### D(4,4)

In [264]:
for c in concepts_for_message(df, [(4,4,4,4,0)]):
    print(c)
## 1 at the second place

(('?', 1, 0, '?'), (0, 1, 1, 0))
((1, 1, 3, 0), (1, 1, 1, 1))
((1, 1, '?', '?'), (1, 1, 0, 0))
((1, 1, 2, '?'), (1, 1, 1, 0))
((1, 1, 0, 1), (1, 1, 1, 1))
((3, 1, 2, 1), (1, 1, 1, 1))
((2, 1, 2, 1), (1, 1, 1, 1))
((3, 1, 2, 3), (1, 1, 1, 1))
((2, 1, '?', '?'), (1, 1, 0, 0))
((1, 1, 2, 2), (1, 1, 1, 1))
((1, 1, '?', 2), (1, 1, 0, 1))
(('?', 1, 3, 0), (0, 1, 1, 1))
((0, 1, 3, 2), (1, 1, 1, 1))
((3, 1, '?', 1), (1, 1, 0, 1))
(('?', 1, 3, 2), (0, 1, 1, 1))
(('?', 1, 2, 0), (0, 1, 1, 1))
((1, 1, 3, '?'), (1, 1, 1, 0))
((2, 1, 2, 0), (1, 1, 1, 1))
((0, 1, 1, 1), (1, 1, 1, 1))
((3, 1, 2, 2), (1, 1, 1, 1))
((2, 1, 2, 2), (1, 1, 1, 1))
((3, 1, '?', '?'), (1, 1, 0, 0))
((2, 1, 1, 0), (1, 1, 1, 1))
((1, 1, 3, 2), (1, 1, 1, 1))
((1, 1, 1, 2), (1, 1, 1, 1))
((2, 1, '?', 0), (1, 1, 0, 1))
((3, 1, 3, 1), (1, 1, 1, 1))
((0, 1, 1, 3), (1, 1, 1, 1))
((3, 1, '?', 2), (1, 1, 0, 1))
((3, 1, 3, 3), (1, 1, 1, 1))
((3, 1, 3, 2), (1, 1, 1, 1))
((1, 1, 0, '?'), (1, 1, 1, 0))
((3, 1, 2, '?'), (1, 1, 1, 0))
((3, 

### Lexicon coarse context condition

In [341]:
import os

# Specify the directory path
directory_path = "analysis/lexica"

# Check if the directory exists
if not os.path.exists(directory_path):
    # If it doesn't exist, create the directory
    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created successfully.")
else:
    print(f"Directory '{directory_path}' already exists.")

Directory 'analysis/lexica' created successfully.


In [48]:
for d in datasets:
    df = pd.read_csv('analysis/messages/'+d+'_standard_granularity_coarse.csv')
 
    concepts = []
    mess = []
    lens = []
    for row in df.itertuples():
        concepts.append(concepts_for_message(df, [row[1]]))
        lens.append(len(concepts_for_message(df, [row[1]])))
        mess.append(row[1])

    res = pd.DataFrame({'message':mess, 'n_concepts': lens })
    res.to_csv('analysis/lexica/counts_'+d+'_granularity_coarse.csv', index=False)


In [50]:
for i, d in enumerate(datasets):
    df = pd.read_csv('analysis/messages/' + d + '_standard_granularity_coarse.csv')
    messages = []
    fixed_attribute_values = []
    symbols = []
    for row in df.itertuples():
        objs, fixed = [], []
        message = row[1]
        #print(message)
        concepts = concepts_for_message(df, [message,])
        message = ast.literal_eval(message)
        symbol = (set(message[:-1]))
        #print(message)
        messages.append(message)
        symbols.append(symbol)
        #print(symbol)
        for obj, f in concepts:
            objs.append(obj)
            fixed.append(f)
        #print(objs)
        #print(fixed)
        idx_fixed_attrs = []
        for n in range(n_attributes[i]):
            nth_elements = [sublist[n] for sublist in fixed]
            if all(e == 1 for e in nth_elements):
                idx_fixed_attrs.append(n)
        #print(idx_fixed_attrs)
        fixed_values = ['_'] * n_attributes[i]  # List of n zeros
        for p in idx_fixed_attrs:  # Directly use p as index
            fixed_values[p] = objs[0][p]
            #print(objs[0][p])
        fixed_attribute_values.append(tuple(fixed_values))
    lexicon = pd.DataFrame({'message': messages, 'symbols': symbols, 'fixed attribute values': fixed_attribute_values})
    lexicon= lexicon.groupby('message', as_index=False).agg({'symbols': lambda x: list(x), 'fixed attribute values': lambda x: list(set().union(*x))})
    lexicon.to_csv('analysis/lexica/'+d+'_granularity_coarse.csv', index=False)


visualisation of messages for given concept

In [389]:
import plotly.express as ex

In [ ]:
def plot_sunburst(dataset, n_samples, )
data = dict()

### Fine context condition

In [ ]:
import torch
import pandas as pd
import random

# Assuming these variables are already defined: datasets, paths, setting, non_default_gran_list, n_epochs, n_values, n_attributes


# Go through all datasets
for i, d in enumerate(datasets):
    path_to_run = paths[i] + '/' + str(setting) + '/granularity_fine' + '/' + str(0) + '/'
    
    path_to_interaction_train = path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0'
    path_to_interaction_val = path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0'
    interaction = torch.load(path_to_interaction_train)
    #print("Interaction train path:", path_to_interaction_train)

    messages = interaction.message.argmax(dim=-1)
    messages = [msg.tolist() for msg in messages]
    sender_input = interaction.sender_input
    #print("Sender input shape:", sender_input.shape)
    n_targets = int(sender_input.shape[1] / 2)
    
    # Get target objects and fixed vectors to re-construct concepts
    target_objects = sender_input[:, :n_targets]
    target_objects = k_hot_to_attributes(target_objects, n_values[i])
    # Concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
    (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
    concepts = list(zip(objects, fixed))
    #print(concepts[0])

   # Collect data for the DataFrame

    mess = []
    concept_reps= []

    for idx, (t_objects, t_fixed) in enumerate(concepts):
        #concept_sample = (tuple(t_objects[0]), tuple(t_fixed))
        concept_sample = (
                list(int(x) for x in t_objects[0]), 
                list(int(x) for x in t_fixed)
            )
        mess.append(tuple(messages[idx]))
        concept_reps.append(create_alternative_representation(concept_sample))

        
        

    # Create the DataFrame
    df = pd.DataFrame({'concepts': concept_reps, 'message': mess})
    # Group by messages and aggregate concepts
    grouped_df = df.groupby('message', as_index=False).agg({'concepts': lambda x: list(x)})

    grouped_df.to_csv('analysis/messages/'+str(d) + '_' + str(setting) + '_granularity_fine.csv', index=False)

    ## create clean dataframe, where each concepts appears only once per row and always with same sample for easier comparison


In [49]:
for d in datasets:
    df = pd.read_csv('analysis/messages/'+d+'_standard_granularity_fine.csv')
 
    concepts = []
    mess = []
    lens = []
    for row in df.itertuples():
        concepts.append(concepts_for_message(df, [row[1]]))
        lens.append(len(concepts_for_message(df, [row[1]])))
        mess.append(row[1])

    res = pd.DataFrame({'message':mess, 'n_concepts': lens })
    res.to_csv('analysis/lexica/counts_'+d+'_granularity_fine.csv', index=False)